# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat talks.tsv



Assessing Waterloo�s first undergraduate course on mental health literacy: Did students� attitudes change over time?	Talk	2020-UWSMHR	University of Waterloo	2020-11-5	"Waterloo, ON"	https://web.archive.org/web/20210411054156/https://uwaterloo.ca/mental-health-wellness/cosmh-conference	"Does completion of a course on MHL have an effect on students� MHL-related attitudes, help-seeking intent, or perceived barriers to care? In a pre-post design, this study suggests that a course dedicated to MHL can have a significant, positive impact of several aspects of MHL in students.  Further, our findings support AHS�s commitment to offering AHS 105 to all students on campus in future terms."	"Purpose: In Winter 2020, the University of Waterloo launched its first undergraduate course on Mental Health Literacy (MHL). AHS 105: Mental Health Literacy was offered through the Faculty of Applied Health Sciences and was available to undergraduate students in all six Faculties. With ethics approval, we c

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description,full_abstract
0,What are recurrent memories about? Understandi...,Talk,2021-CSBBCS,CSBBCS,2021-6-17,"Montr�al, QC",NaN,Researchers debate whether recurrent involunta...,Researchers debate whether recurrent involunta...
1,Assessing Waterloo�s first undergraduate cours...,Talk,2020-UWSMHR,University of Waterloo,2020-11-5,"Waterloo, ON",https://web.archive.org/web/20210411054156/htt...,Does completion of a course on MHL have an eff...,"Purpose: In Winter 2020, the University of Wat..."
2,University of Waterloo�s undergraduate course ...,Talk,2020-CICMH,Centre for Innovation in Campus Mental Health,2020-11-3,"Toronto, ON",NaN,We describe the design and evaluation of the U...,"In this evidence-informed session, we describe..."
3,Anxiety-provoking context scenes can reduce th...,Talk*,2019b-CSBBCS,CSBBCS,2019-6-9,"Waterloo, ON",NaN,Reinstating the same context from encoding dur...,"Reinstating the same context during retrieval,..."
4,From tea cakes to trauma: Bridging gaps betwee...,Talk,2019a-CSBBCS,CSBBCS,2019-6-8,"Waterloo, ON",NaN,We investigated the properties of recurrent IA...,Memories of one�s personal past that come to m...
5,From tea cakes to trauma: Bridging gaps betwee...,Talk,2019-TAMeG,TAMeG,2019-5-14,"Toronto, ON",NaN,Are recurrent memories common in the general p...,NaN
6,Altered working memory capacity for social thr...,Talk,2019-INS,INS,2019-2-23,"New York, NY",NaN,Differences in working memory capacity may lea...,Objective: Differences in working memory capac...
7,Bias in long-term and working memory in indivi...,Talk*,2018-MDRS,MDRS,2018-10-6,"Toronto, ON",NaN,Do individuals with high social anxiety show a...,NaN
8,Social anxiety enhances recognition of task- i...,Talk,2018-TAMeG,TAMeG,2018-5-15,"Toronto, ON",NaN,Are highly socially anxious people easily dist...,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [6]:
!ls ../_talks

18-2018-MDRS.md          20-2020-CICMH.md         2019-6-8-2019a-CSBBCS.md
18-2018-TAMeG.md         20-2020-UWSMHR.md        2019-6-9-2019b-CSBBCS.md
19-2019-INS.md           2018-10-6-2018-MDRS.md   2020-11-3-2020-CICMH.md
19-2019-TAMeG.md         2018-5-15-2018-TAMeG.md  2020-11-5-2020-UWSMHR.md
19-2019a-CSBBCS.md       2019-2-23-2019-INS.md    2021-6-17-2021-CSBBCS.md
19-2019b-CSBBCS.md       2019-5-14-2019-TAMeG.md  21-2021-CSBBCS.md


In [7]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
